In [2]:
import urllib
import json
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from keys import client_id, client_secret, app_id

In [40]:
confirmed_time_data = pd.read_csv('time_series_covid_19_confirmed.csv')
death_time_data = pd.read_csv('time_series_covid_19_deaths.csv')
line_data = pd.read_csv('COVID19_open_line_list.csv')

line_data = line_data.rename({'latitude': 'Lat', 'longitude': 'Long',\
                              'country': 'Country_Region', 'province': 'Province_State'}, axis=1)
confirmed_time_data = confirmed_time_data.rename({'Country/Region': 'Country_Region',\
                                                  'Province/State': 'Province_State'}, axis=1)
death_time_data = confirmed_time_data.rename({'Country/Region': 'Country_Region',\
                                              'Province/State': 'Province_State'}, axis=1)

In [46]:
# confirmed_time_data['daily_tmp']
pd.Series(index=pd.date_range('02/15/2020', periods=38, freq='D'))

2020-02-15   NaN
2020-02-16   NaN
2020-02-17   NaN
2020-02-18   NaN
2020-02-19   NaN
2020-02-20   NaN
2020-02-21   NaN
2020-02-22   NaN
2020-02-23   NaN
2020-02-24   NaN
2020-02-25   NaN
2020-02-26   NaN
2020-02-27   NaN
2020-02-28   NaN
2020-02-29   NaN
2020-03-01   NaN
2020-03-02   NaN
2020-03-03   NaN
2020-03-04   NaN
2020-03-05   NaN
2020-03-06   NaN
2020-03-07   NaN
2020-03-08   NaN
2020-03-09   NaN
2020-03-10   NaN
2020-03-11   NaN
2020-03-12   NaN
2020-03-13   NaN
2020-03-14   NaN
2020-03-15   NaN
2020-03-16   NaN
2020-03-17   NaN
2020-03-18   NaN
2020-03-19   NaN
2020-03-20   NaN
2020-03-21   NaN
2020-03-22   NaN
2020-03-23   NaN
Freq: D, dtype: float64

In [41]:
coords_time = confirmed_time_data.groupby(['Lat', 'Long']).size().index.tolist()
coords_line = line_data.groupby(['Lat', 'Long']).size().index.tolist()

for df in [confirmed_time_data, death_time_data]:
    df['avg_m_tmp'] = np.nan
    df['avg_m_RH'] = np.nan
    df['avg_m_wind'] = np.nan
    df['avg_m_precip'] = np.nan
    df['daily_tmp'] = pd.Series(range(38), index=pd.date_range('02/15/2020', periods=38, freq='D'))
    df['daily_RH'] = pd.Series(index=pd.date_range('02/15/2020', periods=38, freq='D'))
    df['daily_wind'] = pd.Series(index=pd.date_range('02/15/2020', periods=38, freq='D'))
    df['daily_precip'] = pd.Series(index=pd.date_range('02/15/2020', periods=38, freq='D'))
    
#     df['daily_avg'] = pd.DataFrame({
#                                     'avg_d_tmp': np.nan,
#                                     'avg_d_RH': np.nan,
#                                     'avg_d_wind': np.nan,
#                                     'avg_d_precip': np.nan
#                                     },
#                                     index=pd.date_range('02/15/2020', periods=38, freq='D'))

In [960]:
START_TIME = '02/24/2020'
END_TIME = '03/23/2020'
RADIUS = 350
q = 0
log = open('log{0}.txt'.format(datetime.now().strftime('%d%m%Y')), 'w')
for coord in coords:
    q += 1
    lat = coord[0]
    long = coord[1]
    location = data.loc[data['Lat'] == lat]\
                    .loc[data['Long'] == long]\
                     [['Country_Region','Province_State']]
    country = location.iloc[0][0]
    province = location.iloc[0][1]
    url = 'https://api.aerisapi.com/observations/summary/closest?p={0},{1}&from={2}&to={3}&radius={4}miles&plimit=31&limit=1&'\
          'client_id={5}&client_secret={6}'.format(lat, long, START_TIME, END_TIME, RADIUS, client_id, client_secret)
    print('collecting for {0} location {1}, {2}\n{3}'.format(q, province, country, url))
    request = urllib.request.urlopen(url)
    response = request.read()
    json_ = json.loads(response)
    if json_['success']: 
        print('success')
        try:
            for day in json_['response'][0]['periods']:
                date = datetime.strptime(str(day['summary']['ymd']), '%Y%m%d').strftime("%-m/%-d/%Y")
                for index in data.loc[data['Date'] == date]\
                                 .loc[data['Lat'] == coord[0]]\
                                 .loc[data['Long'] == coord[1]].index:
                    data.at[index, 'avg_d_tmp'] = day['summary']['temp']['avgC']
                    data.at[index, 'avg_d_RH'] = day['summary']['rh']['avg']
                    data.at[index, 'avg_d_wind'] = day['summary']['wind']['avgKPH']
                    data.at[index, 'avg_d_precip'] = day['summary']['precip']['totalMM']
#                     data.at[index, 'avg_d_sun'] = day['summary']['solrad']['avgWM2']                    
        except IndexError:
            if json_['error']:
                msg = 'No data found for {0}, {1} at {2}\n{3}\r\n'.format(country, province, date, json_['error'])
                print(msg)
                log.write(msg)
        except Exception:
            msg = 'Unknowen error\n{0}\r\n'.format(json_['error'])
            print(msg)
            log.write(msg)
        finally:
            request.close()
log.close()

collecting for 451 location nan, Slovakia
https://api.aerisapi.com/observations/summary/closest?p=48.669,19.699&from=02/14/2020&to=02/24/2020&radius=350miles&plimit=31&limit=1&client_id=vv2IYjrS6kv2D52249jl9&client_secret=ZbwMRSbTTYKsqBsQRkYyoquHjq2Enr1gkaXHxC33
success
collecting for 452 location Washington, US
https://api.aerisapi.com/observations/summary/closest?p=48.8787,-121.9719&from=02/14/2020&to=02/24/2020&radius=350miles&plimit=31&limit=1&client_id=vv2IYjrS6kv2D52249jl9&client_secret=ZbwMRSbTTYKsqBsQRkYyoquHjq2Enr1gkaXHxC33
success
collecting for 453 location British Columbia, Canada
https://api.aerisapi.com/observations/summary/closest?p=49.2827,-123.1207&from=02/14/2020&to=02/24/2020&radius=350miles&plimit=31&limit=1&client_id=vv2IYjrS6kv2D52249jl9&client_secret=ZbwMRSbTTYKsqBsQRkYyoquHjq2Enr1gkaXHxC33
success
collecting for 454 location Channel Islands, United Kingdom
https://api.aerisapi.com/observations/summary/closest?p=49.3723,-2.3644&from=02/14/2020&to=02/24/2020&radiu

success
collecting for 481 location Northwest Territories, Canada
https://api.aerisapi.com/observations/summary/closest?p=64.8255,-124.8457&from=02/14/2020&to=02/24/2020&radius=350miles&plimit=31&limit=1&client_id=vv2IYjrS6kv2D52249jl9&client_secret=ZbwMRSbTTYKsqBsQRkYyoquHjq2Enr1gkaXHxC33
success
collecting for 482 location nan, Iceland
https://api.aerisapi.com/observations/summary/closest?p=64.9631,-19.0208&from=02/14/2020&to=02/24/2020&radius=350miles&plimit=31&limit=1&client_id=vv2IYjrS6kv2D52249jl9&client_secret=ZbwMRSbTTYKsqBsQRkYyoquHjq2Enr1gkaXHxC33
success
collecting for 483 location Greenland, Denmark
https://api.aerisapi.com/observations/summary/closest?p=71.7069,-42.6043&from=02/14/2020&to=02/24/2020&radius=350miles&plimit=31&limit=1&client_id=vv2IYjrS6kv2D52249jl9&client_secret=ZbwMRSbTTYKsqBsQRkYyoquHjq2Enr1gkaXHxC33
success


In [961]:
############################
# HANDLE NaN AND SAVE DATA #
############################

data = backup.copy()
data[['Province_State']] = data[['Province_State']].fillna(0)
data = data.drop('avg_sun', axis=1)
data = data.dropna()

In [962]:
##############################################
# COMPUTE AVG PARAMETERS FOR CONFIRMED CASES #
##############################################

conf_data = data[data['Case_Type'] == 'Confirmed']
death_data = data[data['Case_Type'] == 'Deaths']
for coord in coords:
    lat = coord[0]
    long = coord[1]
    indices = conf_data.loc[conf_data['Lat'] == lat].loc[conf_data['Long'] == long].index.tolist()
    avg_tmp = conf_data.loc[conf_data['Lat'] == lat]\
                       .loc[conf_data['Long'] == long]['avg_tmp'].mean()
    
    avg_rh = conf_data.loc[conf_data['Lat'] == lat]\
                   .loc[conf_data['Long'] == long]['avg_RH'].mean()
    
    avg_wind = conf_data.loc[conf_data['Lat'] == lat]\
                   .loc[conf_data['Long'] == long]['avg_wind'].mean()
    
    avg_precip = conf_data.loc[conf_data['Lat'] == lat]\
                   .loc[conf_data['Long'] == long]['avg_precip'].mean()                             
    
    for index in indices:
        conf_data.at[index, 'avg_m_tmp'] = avg_tmp
        conf_data.at[index, 'avg_m_RH'] = avg_rh
        conf_data.at[index, 'avg_m_wind'] = avg_wind
        conf_data.at[index, 'avg_m_precip'] = avg_precip                            

In [963]:
###########################################
# COMPUTE AVG PARAMETERS FOR DEATHS CASES #
###########################################

for coord in coords:
    lat = coord[0]
    long = coord[1]
    indices = death_data.loc[data['Lat'] == lat].loc[death_data['Long'] == long].index.tolist()
    avg_tmp = death_data.loc[death_data['Lat'] == lat]\
                       .loc[death_data['Long'] == long]['avg_tmp'].mean()
    
    avg_rh = death_data.loc[death_data['Lat'] == lat]\
                   .loc[death_data['Long'] == long]['avg_RH'].mean()
    
    avg_wind = death_data.loc[death_data['Lat'] == lat]\
                   .loc[death_data['Long'] == long]['avg_wind'].mean()
    
    avg_precip = death_data.loc[death_data['Lat'] == lat]\
                   .loc[death_data['Long'] == long]['avg_precip'].mean()                             
    
    for index in indices:
        death_data.at[index, 'avg_m_tmp'] = avg_tmp
        death_data.at[index, 'avg_m_RH'] = avg_rh
        death_data.at[index, 'avg_m_wind'] = avg_wind
        death_data.at[index, 'avg_m_precip'] = avg_precip                            

In [964]:
#####################################################################
# COMPUTE GROWTH-FACTOR AND MAX CASES, MAX DATE FOR CONFIRMED CASES #
#####################################################################

conf_data['max_date'] = ''
conf_data['growth_factor'] = np.nan
conf_data['max_cases'] = np.nan
for coord in coords:
    lat = coord[0]
    long = coord[1]
    max_cases = conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]['Cases'].max()
    max_date = conf_data.loc[conf_data['Lat'] == lat]\
                        .loc[conf_data['Long'] == long][conf_data['Cases'] == max_cases]['Date']
    
    max_date_parsed = datetime.strptime('3/8/2020', '%m/%d/%Y')
    four_days = max_date_parsed - timedelta(days = 4)
    
    try:
        four_days_cases = conf_data.loc[conf_data['Lat'] == lat]\
                         .loc[conf_data['Long'] == long]\
                         .loc[conf_data['Date'] == four_days]['Cases'].values[0]
    except IndexError:
        four_days_cases = 0
        
    delta = max_cases - four_days_cases 
    if delta <= 0:
        growth_factor = 0
    else:
        growth_factor = delta**0.25
    
    indices = conf_data.loc[conf_data['Lat'] == lat]\
                        .loc[conf_data['Long'] == long].index.tolist()
    for index in indices:
        conf_data.at[index, 'max_date'] = max_date.values[0] 
        conf_data.at[index, 'growth_factor'] = growth_factor
        conf_data.at[index, 'max_cases'] = max_cases

In [965]:
#####################################################################
# COMPUTE GROWTH-FACTOR AND MAX CASES, MAX DATE FOR DEATH CASES #
#####################################################################

death_data['max_date'] = ''
death_data['growth_factor'] = np.nan
death_data['max_cases'] = np.nan
for coord in coords:
    lat = coord[0]
    long = coord[1]
    max_cases = death_data.loc[death_data['Lat'] == lat]\
                         .loc[death_data['Long'] == long]['Cases'].max()
    max_date = death_data.loc[death_data['Lat'] == lat]\
                        .loc[death_data['Long'] == long][death_data['Cases'] == max_cases]['Date']
    
    max_date_parsed = datetime.strptime('3/8/2020', '%m/%d/%Y')
    four_days = max_date_parsed - timedelta(days = 4)
    
    try:
        four_days_cases = death_data.loc[death_data['Lat'] == lat]\
                         .loc[death_data['Long'] == long]\
                         .loc[death_data['Date'] == four_days]['Cases'].values[0]
    except IndexError:
        four_days_cases = 0
        
    delta = max_cases - four_days_cases 
    if delta <= 0:
        growth_factor = 0
    else:
        growth_factor = delta**0.25
    
    indices = death_data.loc[death_data['Lat'] == lat]\
                        .loc[death_data['Long'] == long].index.tolist()
    for index in indices:
        death_data.at[index, 'max_date'] = max_date.values[0] 
        death_data.at[index, 'growth_factor'] = growth_factor
        death_data.at[index, 'max_cases'] = max_cases

In [972]:
######################
# SAVE COMPUTED DATA #
######################

conf_data.to_csv('conf_data.csv')
death_data.to_csv('death_data.csv')


Death cases max_cases correaltion Pearson
Cases            0.999995
Difference       0.855093
growth_factor    0.821659
avg_wind         0.073592
Long             0.062536
Lat              0.061170
avg_RH           0.028359
avg_tmp          0.000887
avg_m_RH        -0.015177
avg_m_tmp       -0.016858
avg_precip      -0.032204
avg_m_wind      -0.054194
avg_m_precip    -0.058316
dtype: float64

Death cases max_cases correaltion Spearman
Cases           0.916256
Difference      0.572851
Lat             0.205643
Long            0.068733
avg_m_RH        0.066074
avg_precip     -0.025254
avg_m_wind     -0.047246
avg_wind       -0.053773
avg_RH         -0.075220
avg_m_precip   -0.089167
avg_tmp        -0.162024
avg_m_tmp      -0.177442
dtype: float64

Confirmed cases max_cases correaltion Pearson
Cases            0.999954
Difference       0.696918
growth_factor    0.679459
Long             0.104040
Lat              0.075724
avg_wind         0.029156
avg_m_RH         0.009652
avg_RH          

In [966]:
############################################################################
# CORRELATIONS AND SCATTER PLOTS AVG TMP VS MAX CASES CONFIRMED AND DEATHS #
############################################################################

TMP_RATE = 1
deaths = death_data[death_data['Date'] == '3/22/2020'][death_data['max_cases'].astype(int) > 0]
confs = conf_data[conf_data['Date'] == '3/22/2020'][conf_data['max_cases'].astype(int) > 0]

deaths['avg_m_tmp'] = deaths['avg_m_tmp'] * TMP_RATE
confs['avg_m_tmp'] = confs['avg_m_tmp'] * TMP_RATE

death_corrs = deaths.corr().unstack().sort_values(kind="quicksort", ascending=False)
death_corrs = death_corrs[death_corrs != 1]
_ = death_corrs.drop_duplicates()
print('\nDeath cases max_cases correaltion Pearson')
print(death_corrs['max_cases'])

death_corrs = deaths.corr('spearman').unstack().sort_values(kind="quicksort", ascending=False)
death_corrs = death_corrs[death_corrs != 1]
_ = death_corrs.drop_duplicates()
print('\nDeath cases max_cases correaltion Spearman')
print(death_corrs['max_cases'])

conf_corrs = confs.corr().unstack().sort_values(kind="quicksort", ascending=False)
conf_corrs = conf_corrs[conf_corrs != 1]
_ = conf_corrs.drop_duplicates()
print('\nConfirmed cases max_cases correaltion Pearson')
print(conf_corrs['max_cases'])

conf_corrs = confs.corr('spearman').unstack().sort_values(kind="quicksort", ascending=False)
conf_corrs = conf_corrs[conf_corrs != 1]
_ = conf_corrs.drop_duplicates()
print('\nConfirmed cases max_cases correaltion Spearman')
print(conf_corrs['max_cases'])

fig, axes = plt.subplots(1, 2, figsize=(15, 7))
_ = axes[0].scatter(conf_data['max_cases'], conf_data['avg_m_tmp'])
_ = axes[1].scatter(death_data['Cases'], death_data['avg_m_tmp'])
_ = axes[0].set_xlabel('max confirmed cases')
_ = axes[0].set_ylabel('avg tmp')
_ = axes[0].set_title('Confirmed cases')
_ = axes[1].set_xlabel('max deaths')
_ = axes[1].set_ylabel('avg tmp')
_ = axes[1].set_title('Death cases')


Confirmed cases growth_rate correaltion Pearson
max_cases       0.725467
Cases           0.725166
Difference      0.653597
Lat             0.293268
Long            0.074156
avg_m_precip    0.034229
avg_precip      0.032194
avg_m_RH        0.029353
avg_RH         -0.039888
avg_wind       -0.047577
avg_m_wind     -0.090362
avg_tmp        -0.253671
avg_m_tmp      -0.260450
dtype: float64

Confirmed cases growth_rate correaltion Spearman
Cases           0.993572
Difference      0.708253
Lat             0.341079
avg_precip      0.053877
Long            0.050958
avg_m_precip    0.043987
avg_m_RH       -0.033968
avg_wind       -0.067642
avg_m_wind     -0.076968
avg_RH         -0.097490
avg_tmp        -0.335001
avg_m_tmp      -0.358204
dtype: float64

Death cases growth_rate correaltion Pearson
max_cases       0.821659
Cases           0.821578
Difference      0.723710
Lat             0.153972
Long            0.081531
avg_m_RH        0.022106
avg_wind        0.016370
avg_RH          0.006595
a

In [967]:
################################################################################
# CORRELATIONS AND SCATTER PLOTS AVG TMP VS GROWTH FACTOR CONFIRMED AND DEATHS #
################################################################################

TMP_RATE = 1
confs = conf_data[conf_data['Date'] == '3/22/2020'][conf_data['Cases'].astype(int) > 0]
deaths = death_data[death_data['Date'] == '3/22/2020'][death_data['max_cases'].astype(int) > 0]
confs['avg_m_tmp'] = confs['avg_m_tmp'] * TMP_RATE
deaths['avg_m_tmp'] = deaths['avg_m_tmp'] * TMP_RATE

conf_corrs_p = confs.corr().unstack().sort_values(kind="quicksort", ascending=False)
conf_corrs_p = conf_corrs_p[conf_corrs_p != 1]
_ = conf_corrs_p.drop_duplicates()
print('\nConfirmed cases growth_rate correaltion Pearson')
print(conf_corrs_p['growth_factor'])

conf_corrs_s = confs.corr('spearman').unstack().sort_values(kind="quicksort", ascending=False)
conf_corrs_s = conf_corrs_s[conf_corrs_s != 1]
_ = conf_corrs_s.drop_duplicates()
print('\nConfirmed cases growth_rate correaltion Spearman')
print(conf_corrs_s['growth_factor'])

death_corrs_p = deaths.corr().unstack().sort_values(kind="quicksort", ascending=False)
death_corrs_p = death_corrs_p[death_corrs_p != 1]
_ = death_corrs_p.drop_duplicates()
print('\nDeath cases growth_rate correaltion Pearson')
print(death_corrs_p['growth_factor'])

death_corrs_s = deaths.corr('spearman').unstack().sort_values(kind="quicksort", ascending=False)
death_corrs_s = death_corrs_s[death_corrs_s != 1]
_ = death_corrs_s.drop_duplicates()
print('\nDeath cases growth_rate correaltion Spearman')
print(death_corrs_s['growth_factor'])

fig, axes = plt.subplots(1, 2, figsize=(15, 7))
_ = axes[0].scatter(conf_data[conf_data['growth_factor'].astype(int) > 0]['growth_factor'],\
                    conf_data[conf_data['growth_factor'].astype(int) > 0]['avg_m_tmp'])

_ = axes[1].scatter(death_data[death_data['growth_factor'].astype(int) > 0]['growth_factor'],\
                    death_data[death_data['growth_factor'].astype(int) > 0]['avg_m_tmp'])

_ = axes[0].set_xlabel('4 days max growth factor')
_ = axes[0].set_ylabel('AVG TMP in last month')
_ = axes[0].set_title('Confirmed cases')
_ = axes[1].set_xlabel('4 days max growth factor')
_ = axes[1].set_ylabel('AVG TMP in last month')
_ = axes[1].set_title('Death cases')